<a href="https://colab.research.google.com/github/LorenzKlose/Capstone/blob/main/Capstone_week_3_part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Part 1

Import libraries

In [62]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup
import urllib.request

Webscraping

In [63]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")

Data modification

In [64]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

Data presentation

In [65]:
df.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Part 2

Data import

In [66]:
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

Merge the two datasets

In [69]:
merged = pd.merge(left=df, right=data, left_on='PostalCode', right_on='Postal Code')
merged.drop(['Postal Code'], axis=1, inplace=True)
merged.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Part 3

Create a datframe with the coordinates for each borough

In [74]:
merged_grouped = merged.groupby('Borough').mean()
merged_grouped

,Latitude,Longitude
Borough,,
Central Toronto,43.701980,-79.398954
Downtown Toronto,43.654624,-79.384184
Downtown Toronto Stn A,43.646435,-79.374846
East Toronto,43.671110,-79.325428
East Toronto Business,43.662744,-79.321558
East York,43.704043,-79.335287
East York/East Toronto,43.685347,-79.338106
Etobicoke,43.655797,-79.537348
Etobicoke Northwest,43.706748,-79.594054


Run the KMeans algorithm with my choose of number of clusters which is 3

In [78]:
kclusters = 3
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(merged_grouped)
kmeans.labels_

array([2, 2, 2, 0, 0, 0, 0, 1, 1, 1, 2, 2, 0, 2, 2], dtype=int32)

Add the cluster belonging to each row in the dataframe

In [95]:
A = list(kmeans.labels_)
merged_grouped['Cluster'] = A
merged_grouped.reset_index(level=0, inplace=True)
merged_grouped.head(10)

,index,Borough,Latitude,Longitude,Cluster
0,0,Central Toronto,43.701980,-79.398954,2
1,1,Downtown Toronto,43.654624,-79.384184,2
2,2,Downtown Toronto Stn A,43.646435,-79.374846,2
3,3,East Toronto,43.671110,-79.325428,0
4,4,East Toronto Business,43.662744,-79.321558,0
5,5,East York,43.704043,-79.335287,0
6,6,East York/East Toronto,43.685347,-79.338106,0
7,7,Etobicoke,43.655797,-79.537348,1
8,8,Etobicoke Northwest,43.706748,-79.594054,1
9,9,Mississauga,43.636966,-79.615819,1


Plot the boroughs and their clusters

In [94]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged_grouped['Latitude'], merged_grouped['Longitude'], merged_grouped['Borough'], merged_grouped['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters